In [18]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 13.1
    Uninstalling websockets-13.1:
      Successfully uninstalled websockets-13.1
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29



  Using cached typer-0.9.4-py3-none-any.whl.metadata (14 kB)
Using cached typer-0.9.4-py3-none-any.whl (45 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.12.5
    Uninstalling typer-0.12.5:
      Successfully uninstalled typer-0.12.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.5.0 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible.


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
from google.colab import userdata
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"]="gpt-3.5-turbo"

## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [6]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the bests teps to invest according to the stock.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
strategic_agent = Agent(
    role="Strategic investment Advisor",
    goal="Evaluate and provide insights on the steps for investing"
         "and mitigation strategy associated with potential trading activities.",
    backstory="Armed with a deep understanding of investment assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "investment pipeline of trades. It offers a detailed analysis of "
              "steps to invest and suggests safeguards to ensure that "
              "investment steps result in profitability.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [12]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential steps on how to invest for {stock_selection} "
        "that align with the user's risk tolerance{risk_tolerance}."
    ),
    agent=trading_strategy_agent,
)


In [13]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for investing in {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to"
        "invest in trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [14]:
# Task for Risk Advisor Agent: Assess Trading Risks
strategic_task = Task(
    description=(
        "Evaluate the risks associated with the proposed steps "
        "strategies and execution plans for investing in {stock_selection}. "
        "Provide a detailed analysis of steps on how to invest and when."
    ),
    expected_output=(
        "A comprehensive investment analysis report detailing potential "
        "investment steps and mitigation recommendations for {stock_selection}."
    ),
    agent=strategic_agent,
)

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            strategic_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           strategic_task],

    manager_llm=ChatOpenAI(model="gpt-3.5-turbo",
                           temperature=0.8),
    process=Process.hierarchical,
    verbose=True
)

In [16]:
financial_trading_inputs = {
    'stock_selection': 'Reliance JIO',
    'initial_capital': '20000 rupees',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [17]:
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (Reliance JIO). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for Reliance JIO to the Data Analyst, as they have the expertise in statistical modeling and machine learning required for this task.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for Reliance JIO using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for Reliance JIO."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for Reliance JIO to identify trends and predict market movements.

Action: Read website content
Action Input: {"website_url": 

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
Markdown(result)

As the Trading Strategy Developer, I have developed detailed trading strategies for investing in Reliance Jio based on the current market conditions and upcoming events. Considering the upcoming IPO in 2025, the declining stock price, negative sentiment, and expected quarterly results, the following execution plans are suggested:

1. Short-term trading opportunities: Given the recent stock price movements and negative sentiment, it may be beneficial to consider short-term trading opportunities to capitalize on market fluctuations.

2. Monitor IPO developments: Closely monitoring the progress and announcements related to the upcoming IPO in 2025 will provide valuable insights into the company's future prospects and stock price movements.

3. Quarterly results analysis: Analyzing the quarterly results for Reliance Jio to identify trends in net profit and net sales will help in making informed trading decisions based on the company's financial performance.

4. Risk management strategies: Developing risk management strategies to mitigate potential losses and protect investments in volatile market conditions will be crucial for successful trading in Reliance Jio.

By aligning these strategies with the user's risk tolerance and leveraging insights from the Data Analyst, we can make informed investment decisions for Reliance Jio that optimize returns and minimize risks.

In [20]:
def analyze_investment_with_updates(stock_selection, initial_capital, risk_tolerance, trading_strategy_preference, news_impact_consideration):
    inputs = {
        'stock_selection': stock_selection,
        'initial_capital': initial_capital,
        'risk_tolerance': risk_tolerance,
        'trading_strategy_preference': trading_strategy_preference,
        'news_impact_consideration': news_impact_consideration
    }

    # This will store real-time updates
    steps = []

    def callback(message):
        steps.append(message)
        return steps, "Thinking..."

    # Simulate a real-time process by updating steps
    for task in financial_trading_crew.tasks:
        agent_thoughts = f"Agent {task.agent.role} is executing: {task.description.format(**inputs)}"
        callback(agent_thoughts)

    # Get final result
    result = financial_trading_crew.kickoff(inputs=inputs)
    final_output = f"Final Analysis:\n{result}"

    return "\n".join(steps), final_output

In [21]:
iface = gr.Interface(
    fn=analyze_investment_with_updates,
    inputs=[
        gr.Textbox(label="Stock Selection", placeholder="e.g., Reliance JIO"),
        gr.Textbox(label="Initial Capital", placeholder="e.g., 20000 rupees"),
        gr.Dropdown(label="Risk Tolerance", choices=["Low", "Medium", "High"]),
        gr.Dropdown(label="Trading Strategy Preference", choices=["Day Trading", "Swing Trading", "Long Term"]),
        gr.Checkbox(label="Consider News Impact?")
    ],
    outputs=[
        gr.Textbox(label="Agent Thoughts (Real-Time)", lines=10, interactive=False),
        gr.Textbox(label="Final Investment Analysis", lines=10, interactive=False)
    ],
    title="Multi AI Agent Investment Analysis with Real-Time Updates",
    description="Monitor real-time agent thinking and execution, followed by a comprehensive investment analysis."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://516e936cde64223a58.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [26]:
def analyze_investment_with_complete_flow(stock_selection, initial_capital, risk_tolerance, trading_strategy_preference, news_impact_consideration):
    inputs = {
        'stock_selection': stock_selection,
        'initial_capital': initial_capital,
        'risk_tolerance': risk_tolerance,
        'trading_strategy_preference': trading_strategy_preference,
        'news_impact_consideration': news_impact_consideration
    }

    flow = []

    # Simulate agentic actions (for demonstration, this would normally be part of the crew processing)
    for task in financial_trading_crew.tasks:
        agent_thoughts = f"## {task.agent.role} Thoughts:\n**Executing:** {task.description.format(**inputs)}\n"
        flow.append(agent_thoughts)
        flow.append(f"**Interim Output:** {task.expected_output.format(**inputs)}\n")

    # Run the crew process
    result = financial_trading_crew.kickoff(inputs=inputs)

    # Join all the steps for agentic flow
    agentic_flow_output = "\n".join(flow)

    # Final result as markdown
    final_output = f"## Final Investment Analysis\n{result}"

    return agentic_flow_output, final_output

In [29]:
iface = gr.Interface(
    fn=analyze_investment_with_complete_flow,
    inputs=[
        gr.Textbox(label="Stock Selection", placeholder="e.g., Reliance JIO"),
        gr.Textbox(label="Initial Capital", placeholder="e.g., 20000 rupees"),
        gr.Dropdown(label="Risk Tolerance", choices=["Low", "Medium", "High"]),
        gr.Dropdown(label="Trading Strategy Preference", choices=["Day Trading", "Swing Trading", "Long Term"]),
        gr.Checkbox(label="Consider News Impact?")
    ],
    outputs=[
        gr.Textbox(label="Agentic Flow (Detailed Execution)", lines=15, interactive=False),
        gr.Markdown(label="Final Investment Analysis")
    ],
    title="Investment Analysis with Agentic Flow",
    description="View the entire process executed by agents and the final investment strategy in two distinct sections."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://26e8b0781d88fe99d8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
